In [ ]:
#install streamlit, one time execution
!pip install streamlit

In [ ]:
#import streamlit
import streamlit as st

In [ ]:
!pip install st-annotated-text

In [ ]:
!pip install pyngrok

In [ ]:
from __future__ import print_function

from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import os


class GoogleDriveService:
    def __init__(self):
        self._SCOPES=['https://www.googleapis.com/auth/drive']

        _base_path = os.path.dirname('/content/')
        _credential_path=os.path.join(_base_path, 'credential.json')
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = _credential_path

    def build(self):
        creds = ServiceAccountCredentials.from_json_keyfile_name(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"), self._SCOPES)
        service = build('drive', 'v3', credentials=creds)

        return service

def create_basic(fid, name, path):
    'fid: the folder id of drive under which you would create a new file.'
    try:
        file_metadata = {'name':name, 'parents': [fid]}
        media = MediaFileUpload(path,
                                mimetype='application/x-hdf5')
        # pylint: disable=maybe-no-member
        file = g_drive_service.files().create(body=file_metadata, media_body=media,
                                      fields='id').execute()
        print(F'File ID: {file.get("id")}')

    except HttpError as error:
        print(F'An error occurred: {error}')
        file = None

    return file.get('id')

def update_basic(fid, name, path):
    'fid: the id of the drive file you want to update'
    try:
        media = MediaFileUpload(path, mimetype='application/x-hdf5')
        # pylint: disable=maybe-no-member
        file_metadata = {'name': name}
        file = g_drive_service.files().update(fileId=fid,body=file_metadata, media_body=media).execute()
        print(F'File ID: {file.get("id")}')

    except HttpError as error:
        print(F'An error occurred: {error}')
        file = None

    return file.get('id')

selected_fields="files(id,name,webViewLink)"
g_drive_service=GoogleDriveService().build()
list_file=g_drive_service.files().list(fields=selected_fields).execute()
print(list_file)

folder_id='1pKC2WaM0UxeBIS_d05kP5TC2wKavuIo9'

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten,Dropout, BatchNormalization
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image
import collections
from matplotlib import pyplot as plt
class SimpleMLP:
    @staticmethod
    def build():
        model = Sequential() 
        model.add(Conv2D(filters=16, kernel_size= (3,3), activation= 'relu', input_shape=(66,66,3)) )
        model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu' ))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(BatchNormalization())
        model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu' ))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu' ))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(rate=0.25))
        model.add(Flatten())
        model.add(Dense(units=256, activation='relu'))
        model.add(Dense(units=128, activation='relu'))
        model.add(Dropout(rate=0.25))
        model.add(Dense(units=5, activation='sigmoid'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        
        return model



In [ ]:
#@title initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build()
global_model.save('global_model.h5')
path = "/content/global_model.h5"
global_update_id=create_basic(folder_id,'global_model.h5', path) # initial copy of global model is created. 

File ID: 1oBHRty1v3-ff3QoiF3M433zXmIBWwt07


In [ ]:
#@title initialize local model 1
smlp_global = SimpleMLP()
local_model = smlp_global.build()
local_model.save('local_model_1.h5')
path = "/content/local_model_1.h5"
local1_update_id=create_basic(folder_id,'local_model_1.h5', path) # initial copy of local model is created. 

File ID: 16Ui-Qmq5FDOBA1lPOhoOqze0ohf-WSIC


In [ ]:
#@title initialize local model 2
smlp_global = SimpleMLP()
local_model = smlp_global.build()
local_model.save('local_model_2.h5')
path = "/content/local_model_2.h5"
local2_update_id=create_basic(folder_id, 'local_model_2.h5', path) # initial copy of local model is created. 

File ID: 10BzSLkBET62es9looHQ6C2JXcfc6kOsi


In [ ]:
#@title initialize local model 3
smlp_global = SimpleMLP()
local_model = smlp_global.build()
local_model.save('local_model_3.h5')
path = "/content/local_model_3.h5"
local3_update_id=create_basic(folder_id, 'local_model_3.h5', path) # initial copy of local model is created. 

File ID: 1XVHi9C3nwaw5Op0sOeOHGVe17qdGs-Fx


In [ ]:
%%writefile app.py
#@title app.py
from __future__ import print_function





import keras
import keras.utils
from keras.layers import Dense,Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization
from keras.models import Sequential
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras import utils as np_utils
from annotated_text import annotated_text
from tensorflow.keras.optimizers import SGD

import streamlit as st
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten,Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image
import collections
from matplotlib import pyplot as plt

from keras.preprocessing import image
import zipfile
import cv2
import os
import shutil 
import math
import random
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import streamlit as st
from io import StringIO
import shutil
import os
from PIL import Image 

###############################################################################################
global_update_id = '1oBHRty1v3-ff3QoiF3M433zXmIBWwt07' 
local1_update_id = '16Ui-Qmq5FDOBA1lPOhoOqze0ohf-WSIC'
local2_update_id = '10BzSLkBET62es9looHQ6C2JXcfc6kOsi'
local3_update_id = '1XVHi9C3nwaw5Op0sOeOHGVe17qdGs-Fx'
locals_upid = [ local1_update_id, local2_update_id, local3_update_id ]

lr = 0.001
comms_round = 40
loss='categorical_crossentropy'
optimizer =  tf.keras.optimizers.legacy.SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               )  
metrics = ['accuracy'] # f1_score, precision_score

###############################################################################################


#@title downloading weights from drive
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)



#@title for showing progress of model training
class CustomCallback(keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        keys = list(logs.keys())
        print("Starting training; got log keys: {}".format(keys))

    def on_train_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop training; got log keys: {}".format(keys))

    def on_epoch_begin(self, epoch, logs=None):
        # st.write("Epoch "+str(epoch)+" starts")
        keys = list(logs.keys())
        print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_epoch_end(self, epoch, logs=None):
        st.write("----Epoch----"+str(epoch)+" ends" + "----Accuracy----"+str(logs["accuracy"])+"----Loss----"+ str(logs["loss"])) 
        #+ "----Validation Acc----"+str(logs["val_accuracy"]) + "----Validation Loss----"+str(logs["val_loss"]))
        keys = list(logs.keys())
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_test_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start testing; got log keys: {}".format(keys))

    def on_test_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop testing; got log keys: {}".format(keys))

    def on_predict_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start predicting; got log keys: {}".format(keys))

    def on_predict_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop predicting; got log keys: {}".format(keys))

    def on_train_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: start of batch {}; got log keys: {}".format(batch, keys))

    def on_train_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: end of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: start of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: end of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: start of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: end of batch {}; got log keys: {}".format(batch, keys))
    




#@title broken pipe error solve
from requests.exceptions import ConnectionError

def retry_on_connectionerror(f,id,updated_by_who, path, max_retries=5):
  retries = 0
  while retries < max_retries:
    try:
      return f(id, updated_by_who, path)
    except ConnectionError:
      retries += 1
      st.write("retries: "+retries)
  raise Exception("Maximum retries exceeded")

##############################################################################

class GoogleDriveService:
    def __init__(self):
        self._SCOPES=['https://www.googleapis.com/auth/drive']

        _base_path = os.path.dirname('/content/')
        _credential_path=os.path.join(_base_path, 'credential.json')
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = _credential_path

    def build(self):
        creds = ServiceAccountCredentials.from_json_keyfile_name(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"), self._SCOPES)
        service = build('drive', 'v3', credentials=creds)

        return service

def create_basic(fid, name, path):
    'fid: the folder id of drive under which you would create a new file.'
    try:
        file_metadata = {'name':name, 'parents': [fid]}
        media = MediaFileUpload(path,
                                mimetype='application/x-hdf5')
        # pylint: disable=maybe-no-member
        file = g_drive_service.files().create(body=file_metadata, media_body=media,
                                      fields='id').execute()
        print(F'File ID: {file.get("id")}')

    except HttpError as error:
        print(F'An error occurred: {error}')
        file = None

    return file.get('id')

def update_basic(fid, name, path):
    'fid: the id of the drive file you want to update'
    try:
        media = MediaFileUpload(path, mimetype='application/x-hdf5')
        # pylint: disable=maybe-no-member
        file_metadata = {'name': name}
        file = g_drive_service.files().update(fileId=fid,body=file_metadata, media_body=media).execute()
        print(F'File ID: {file.get("id")}')

    except HttpError as error:
        print(F'An error occurred: {error}')
        file = None

    return file.get('id')

g_drive_service=GoogleDriveService().build()

#@title global model initialization

# Model architecture: MLP
class SimpleMLP:
    @staticmethod
    def build():
        model = Sequential() 
        model.add(Conv2D(filters=16, kernel_size= (3,3), activation= 'relu', input_shape=(66,66,3)) )
        model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu' ))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(BatchNormalization())
        model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu' ))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu' ))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(rate=0.25))
        model.add(Flatten())
        model.add(Dense(units=256, activation='relu'))
        model.add(Dense(units=128, activation='relu'))
        model.add(Dropout(rate=0.25))
        model.add(Dense(units=5, activation='sigmoid'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        
        return model


#@title federated learning main algorithm

# Define a function for federated averaging
def federated_averaging(client_model, centralized_model):
    client_weights = client_model.get_weights()
    central_weights = centralized_model.get_weights()
    for i in range(len(central_weights)):
        central_weights[i] = (central_weights[i] + client_weights[i])/2
    centralized_model.set_weights(central_weights)
    return centralized_model


#@title training clients

def train_local_models(client, global_model, client_drive_id):
    # Adding Model check point Callback
    mc = ModelCheckpoint(filepath="/content/cervical_cancer_best_model"+"_updated_by_"+client+".h5", save_freq="epoch", monitor= 'val_accuracy', verbose= 1, save_best_only= True, mode = 'auto');
    call_back = [ mc ];
    global_weights = global_model.get_weights()
    #loop through each client and create new local model
    # for client in client_names:
    st.write("Client_"+client+" is training now")
    
    # smlp_local = SimpleMLP()
    # local_model = smlp_local.build()
    # local_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    file_id = client_drive_id
    path='local_model_received_by_'+client+'.h5'
    download_file_from_google_drive(file_id, path)
    local_model = load_model(path)
    
    #set local model weight to the weight of the global model
    # local_model.set_weights(global_weights)
    'ekhane jokhn ekmatro tokhoni global model weight set korbo'
    local_model.fit(client_train_batched[client],steps_per_epoch=3, epochs=50, validation_data= client_val_batched[client], 
                  validation_steps= 6, verbose=2, callbacks=call_back)
    local_model = load_model("/content/cervical_cancer_best_model"+"_updated_by_"+client+".h5")

    return local_model


## frontend code starts

st.markdown(
         f"""
         <style>
         [data-testid="stSidebar"] > div:first-child {{
             background: url("https://miro.medium.com/max/1024/1*4nKg-FASHe3XW50hHexYDQ.png");
             background-attachment: fixed;
         }}
         </style>
         """,
         unsafe_allow_html=True
     )

with st.sidebar:
    original_title = '<p style="color:Black; font-size: 35px; font-weight: bold">Federated Learning Aided Cervical Cancer Classification WebApp</p>'
    st.markdown(original_title, unsafe_allow_html=True)
    page = st.sidebar.selectbox( 'Select User Type',['Client','Admin Aggregator']) 

if page=='Client':
    #############################################################################################
    
    st.subheader("Step 0")  
    annotated_text(
        ("Enter the ID of your hospital: i.e. 1,2 or 3", "Step 0"),
    )
    #adding a single-line text input widget
    client_name = st.text_input("",'0')
    #displaying the entered text
    st.write('Thank you for your input, the ID number of this client device is', int(client_name)) 
    st.markdown("---------------------------------")

    #############################################################################################

    #############################################################################################

    st.subheader("Step 1")  
    annotated_text(
        ("Request for currently updated Global Model", "Step 1"),
    )
    st.markdown("")  
    global_model_requested = st.button('Request Global Model From Server')
    if global_model_requested:
      file_id = global_update_id
      destination = '/content/global_model.h5'
      download_file_from_google_drive(file_id, destination)
      global_model = load_model(destination)
      st.write("Global model received from server in the following path", destination)
    st.markdown("---------------------------------")  

    #############################################################################################

    #############################################################################################

    st.subheader("Step 2") 
    annotated_text(
        ("Please upload a zip file of your train images:", "Step 2"),
    )
    st.markdown("")  

    if(os.path.isdir('/content/content/Client_'+client_name)==True):
          shutil.rmtree('/content/content/Client_'+client_name)
    uploaded_file = st.file_uploader("Choose a file",type="zip")
    
    if uploaded_file is not None:
      with zipfile.ZipFile(uploaded_file, "r") as z:
        z.extractall(".")
        st.write("Hurray! Your dataset has been successfully uploaded.")
    st.markdown("---------------------------------") 

    #############################################################################################

    #############################################################################################

    st.subheader("Step 3") 
    annotated_text(
        ("Train your local model and save as .h5 format locally", "Step 3"),
    )
    st.markdown("") 
    train = st.button('Train')
    if train:
        st.write('Client '+client_name+' started training now.')
       
        # process and batch the training data for each client
        # client_name format: clients_1
        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(zoom_range = 0.2, shear_range = 0.2 , rescale = 1./255 , horizontal_flip=True)
        train_data = train_datagen.flow_from_directory(directory= "/content/Client_"+client_name, target_size=(66, 66), batch_size=10, class_mode = 'categorical')
        
        # fetch the global model for this round for one client
        global_model = load_model('/content/global_model.h5')
        global_weights = global_model.get_weights()
       
        # set weights of local model to global model's weights and 
        smlp_local = SimpleMLP()
        local_model = smlp_local.build()
        local_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        local_model.fit(train_data, steps_per_epoch=100, epochs=10, verbose=1, callbacks=[CustomCallback()])
        
        local_model_name = 'local_model_'+ client_name + '.h5'
        path = "/content/"+ local_model_name
        local_model.save(path)
        
        
    st.markdown("---------------------------------") 
    
    #############################################################################################

    #############################################################################################

    st.subheader("Step 4") 
    annotated_text(
        ("Send your locally trained parameters to the server, your data would be safe with yourself.", "Step 4"),
    )
    st.markdown("") 
    send = st.button('Send')
    
    if send:
      local_model_name = 'local_model_'+ client_name + '.h5'
      path = "/content/"+ local_model_name
      id = update_basic(locals_upid[int(client_name)-1], local_model_name, path)
      st.write("Successfully sent the parameters to the server.")
   
    
    
    st.markdown("---------------------------------") 
    
    #############################################################################################

    #############################################################################################

    st.subheader("Step 5") 
    annotated_text(
        ("Upload a test image and see the output of the class and accuracy.", "Step 5"),
    )
    st.markdown("") 

    uploaded_file = st.file_uploader("Choose a test image file",type="zip")
    
    if uploaded_file is not None:
      with zipfile.ZipFile(uploaded_file, "r") as z:
        z.extractall(".")
        st.write("Hurray! Your test dataset has been successfully uploaded.")

    test = st.button('Test')
    if test:      
        local_model_name = 'local_model_'+ client_name + '.h5'
        path = "/content/"+ local_model_name
        local_model = load_model(path)
        
        test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
        test_data = test_datagen.flow_from_directory(directory= "/content/test/", target_size=(66, 66), batch_size=10, class_mode = 'categorical')

        accuracy1 = local_model.evaluate_generator(generator= test_data)[1] 

        file_id = global_update_id
        destination = '/content/global_model.h5'
        download_file_from_google_drive(file_id, destination)
        global_model = load_model(destination)

        accuracy2 = global_model.evaluate_generator(generator= test_data)[1] 

        st.write("The accuracy of your local model is = "+ str(accuracy1*100) +" %")
        st.write("The accuracy of the global model is = "+ str(accuracy2*100) +" %")

    st.markdown("---------------------------------") 

if page=='Admin Aggregator':
    
    st.subheader("Step 0")  
    annotated_text(
        ("Enter the admin credential ID", "Step 0"),
    )
    #adding a single-line text input widget
    admin_name = st.text_input("",'0')
    #displaying the entered text
    if(admin_name=="admin"):
      st.write('Admin Verified')
      st.subheader("Step 1") 
      annotated_text(
          ("Please aggregate the local updates with the global model", "Step 1"),
      )
      st.markdown("") 
      aggre = st.button('Aggregate') 
      if aggre:
        file_id = local1_update_id
        destination = '/content/local_model_1.h5'
        download_file_from_google_drive(file_id, destination)
        local1 = load_model(destination)
        st.write("Local model received from server in the following path", destination)

        file_id = local2_update_id
        destination = '/content/local_model_2.h5'
        download_file_from_google_drive(file_id, destination)
        local2 = load_model(destination)
        st.write("Local model received from server in the following path", destination)

        file_id = local3_update_id
        destination = '/content/local_model_3.h5'
        download_file_from_google_drive(file_id, destination)
        local3 = load_model(destination)
        st.write("Local model received from server in the following path", destination)

        
        locals = [local1, local2, local3]
        
        
        # fetch the global model for this round for one client
        global_model = load_model('/content/global_model.h5')
        global_weights = global_model.get_weights()
       
        # set weights of local model to global model's weights and 
        smlp_local = SimpleMLP()
        temp_model = smlp_local.build()
        temp_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        
        #set local model weight to the weight of the global model
        temp_model.set_weights(global_weights)

        temp_model = federated_averaging(local1,temp_model)
        temp_model = federated_averaging(local2,temp_model)
        temp_model = federated_averaging(local3,temp_model)

        path = '/content/global_model.h5'
        temp_model.save(path)
        id = update_basic(global_update_id, 'global_model.h5', path)
        st.write("Successfully aggregated the parameters to the global model.")
         
   



Overwriting app.py


In [ ]:
from pyngrok import ngrok

In [ ]:
ngrok.set_auth_token("2GJ3XbpGRoKhs9VWiuCBUxlPuRM_7veAjgLij9ne349wXFQjg") #ngrok.com

INFO:pyngrok.process:Updating authtoken for default "config_path" of "ngrok_path": /usr/local/lib/python3.8/dist-packages/pyngrok/bin/ngrok
2023-02-25 21:23:39.581 INFO    pyngrok.process: Updating authtoken for default "config_path" of "ngrok_path": /usr/local/lib/python3.8/dist-packages/pyngrok/bin/ngrok


In [ ]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(port = '80')
print(url)

In [ ]:
tunnels = ngrok.get_tunnels()
tunnels

# try:
#     # Block until CTRL-C or some other terminating event
#     ngrok_process.proc.wait()
# except KeyboardInterrupt:
#     print(" Shutting down server.")

#     ngrok.kill()

INFO:pyngrok.process.ngrok:t=2023-02-24T16:15:49+0000 lvl=info msg=start pg=/api/tunnels id=1dbb1098a4b06a4e
2023-02-24 16:15:49.907 INFO    pyngrok.process.ngrok: t=2023-02-24T16:15:49+0000 lvl=info msg=start pg=/api/tunnels id=1dbb1098a4b06a4e
INFO:pyngrok.process.ngrok:t=2023-02-24T16:15:49+0000 lvl=info msg=end pg=/api/tunnels id=1dbb1098a4b06a4e status=200 dur=253.622µs
2023-02-24 16:15:49.911 INFO    pyngrok.process.ngrok: t=2023-02-24T16:15:49+0000 lvl=info msg=end pg=/api/tunnels id=1dbb1098a4b06a4e status=200 dur=253.622µs


[<NgrokTunnel: "https://0af6-34-73-195-65.ngrok.io" -> "http://localhost:80">,
 <NgrokTunnel: "http://0af6-34-73-195-65.ngrok.io" -> "http://localhost:80">]

In [ ]:
ngrok.kill()

INFO:pyngrok.process:Killing ngrok process: 1568
2023-02-24 16:16:01.919 INFO    pyngrok.process: Killing ngrok process: 1568
